In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pickle
import matplotlib.pyplot as plt

In [2]:
def split_scaler(indep_x,dep_y):
    x_train,x_test,y_train,y_test = train_test_split(indep_x,dep_y,test_size = 0.2,random_state = 0)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def R2_prediction(regressor,x_test,y_test):
    y_pred = regressor.predict(x_test)

    from sklearn.metrics import r2_score
    R2_score = r2_score(y_test,y_pred)
    return R2_score

def linear(x_train,y_train,x_test):
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(x_train,y_train)
    R2_score = R2_prediction(regressor,x_test,y_test)
    return R2_score

def decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(x_train,y_train)
    R2_score = R2_prediction(regressor,x_test,y_test)
    return R2_score    

def random(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 10, random_state=0)
    regressor.fit(x_train,y_train)
    R2_score = R2_prediction(regressor,x_test,y_test)
    return R2_score

def xgboost(x_train,y_train_x_test):
    from xgboost import XGBRegressor
    regressor = XGBRegressor(n_jobs=5, learning_rate=0.01, max_depth=10, randon_state=1)
    regressor.fit(x_train,y_train)
    R2 = R2_prediction(regressor,x_test,y_test)
    return R2_score

In [3]:
def selectkbest(indep_x,dep_y,n):
    kbest_list = []
    cols_list = []
    R2_values = []
   
    kbest = SelectKBest(score_func=chi2, k=n)
    kbest.fit(indep_x,dep_y)
    kbest_features = kbest.transform(indep_x)
    kbest_list.append(kbest_features)

    selected_features = [col for col, selected in zip(indep_x.columns, kbest.get_support()) if selected]
    cols_list.append(selected_features)

    from sklearn.linear_model import LinearRegression
    lin_model = LinearRegression()

    from sklearn.tree import DecisionTreeRegressor
    dc_model = DecisionTreeRegressor(random_state=0)

    from sklearn.ensemble import RandomForestRegressor
    rf_model = RandomForestRegressor(n_estimators=10, random_state=0)

    from xgboost import XGBRegressor
    xgb_model = XGBRegressor(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)

    modellist = [lin_model, dc_model, rf_model, xgb_model]

    for model in modellist:
        x_train,x_test,y_train,y_test = split_scaler(pd.DataFrame(kbest_features), dep_y)
        model.fit(x_train,y_train)
        R2_score = R2_prediction(model,x_test,y_test)
        R2_values.append(R2_score)
    
    
    return kbest_list, cols_list, R2_values

In [4]:
dataset = pd.read_csv("CKD.csv",index_col=None)
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [5]:
df = dataset

In [6]:
df = pd.get_dummies(df,dtype=int,drop_first=True)

In [7]:
indep_x = df.drop('classification_yes',axis=1)
dep_y = df['classification_yes']

In [9]:
kbest_list, cols_list, R2_values = selectkbest(indep_x,dep_y,5)

for model_name, selected_features, R2_values  in zip(["Logistic", "Decision", "Random", "XGBoost"], cols_list, R2_values):
    
    print(f"Model: {model_name}")
    print("Selected Features:", selected_features)
    print(f"R2_values: {R2_values}\n")

Model: Logistic
Selected Features: ['bgr', 'bu', 'sc', 'pcv', 'wc']
R2_values: 0.5815111848406156

